In [1]:
import pandas as pd

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df

,Lugar de residencia,Medida,Valor
0,Huelva,Población,510743
1,Huelva,Confirmados PDIA,6217
2,Huelva,Confirmados PDIA 14 días,2469
3,Huelva,Tasa PDIA 14 días,"483,41338011485226"
4,Huelva,Confirmados PDIA 7 días,1059
...,...,...,...
627,Municipio de Huelva sin especificar,Tasa PDIA 14 días,-
628,Municipio de Huelva sin especificar,Confirmados PDIA 7 días,28
629,Municipio de Huelva sin especificar,Total Confirmados,125
630,Municipio de Huelva sin especificar,Curados,21


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  6217.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  1684.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 460 personas en los últimos 7 días 

Un positivo PCR cada 213 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva-Costa,3195.0,599.0,1290.0,288115.0,207.903094,447.737882,106.0
Huelva,6217.0,1059.0,2469.0,510743.0,207.344986,483.413380,101.0
Huelva (capital),1684.0,312.0,674.0,143663.0,217.174916,469.153505,63.0
Condado-Campiña,1946.0,252.0,752.0,155057.0,162.520879,484.982942,39.0
Sierra de Huelva-Andévalo Central,952.0,180.0,363.0,67571.0,266.386468,537.212710,24.0
Lepe,492.0,89.0,217.0,27431.0,324.450439,791.075790,14.0
Aracena,237.0,85.0,146.0,8107.0,1048.476625,1800.912791,12.0
Cartaya,161.0,44.0,69.0,19974.0,220.286372,345.449084,9.0
Gibraleón,271.0,59.0,149.0,12607.0,467.993972,1181.883081,5.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Escacena del Campo,84.0,6.0,42.0,2284.0,262.697023,1838.879159,0.0
Aracena,237.0,85.0,146.0,8107.0,1048.476625,1800.912791,12.0
Gibraleón,271.0,59.0,149.0,12607.0,467.993972,1181.883081,5.0
Palma del Condado (La),282.0,33.0,102.0,10761.0,306.662950,947.867299,1.0
Villarrasa,69.0,3.0,19.0,2176.0,137.867647,873.161765,1.0
Lepe,492.0,89.0,217.0,27431.0,324.450439,791.075790,14.0
Manzanilla,41.0,5.0,16.0,2135.0,234.192037,749.414520,0.0
Niebla,62.0,10.0,29.0,4117.0,242.895312,704.396405,5.0
Villablanca,29.0,16.0,19.0,2848.0,561.797753,667.134831,4.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Cortegana,123.0,1.0,11.0,4666.0,21.431633,235.747964,1.0,0.090909
Escacena del Campo,84.0,6.0,42.0,2284.0,262.697023,1838.879159,0.0,0.142857
Villarrasa,69.0,3.0,19.0,2176.0,137.867647,873.161765,1.0,0.157895
Alosno,35.0,4.0,23.0,3923.0,101.962784,586.286006,0.0,0.173913
Trigueros,98.0,7.0,39.0,7713.0,90.755867,505.639829,1.0,0.179487
Almonaster la Real,30.0,2.0,9.0,1819.0,109.950522,494.777350,1.0,0.222222
Almonte,304.0,29.0,119.0,24191.0,119.879294,491.918482,3.0,0.243697
Villalba del Alcor,75.0,6.0,22.0,3338.0,179.748352,659.077292,0.0,0.272727
Calañas,34.0,3.0,10.0,2791.0,107.488355,358.294518,0.0,0.300000
